In [ ]:
from transformers import AutoTokenizer, BertModel
import torch
import torch.nn as nn
from torchcrf import CRF
import torch.autograd as autograd
import torch.optim as optim

# Initialize the tokenizer for embeddings
bert_model = BertModel.from_pretrained('bert-base-uncased')


In [ ]:
# construct the model

class BiLSTM_CRF(nn.Module):
    def __init__(self, embedding_dim, tag_to_ix, hidden_dim, num_layers=1, batch_first=True, dropout=0.1):
        super(BiLSTMCrf, self).__init__()
        bert_model = 'bert-base-uncased'
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.tag_to_ix = tag_to_ix
        self.target_size = len(tag_to_ix)

        self.bert_embeds = BertModel.from_pretrained(bert_model)
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim//2, num_layers=1, batch_first=batch_first, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.hidden2tag = nn.Linear(hidden_dim, self.target_size)
        self.crf = CRF(self.target_size, batch_first=True)

    def forward(self, input_ids, labels=None):
        embeds = self.bert(input_ids)[0]
        sequence_output = self.dropout(embeds)
        # h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size) # 2 for bidirectional
        # c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size)
        
        # Forward propagate LSTM
        lstm_out, (hn, cn) = self.lstm(sequence_output, None)
        lstm_feats = self.hidden2tag(lstm_out)
        
        if labels is not None:
            loss = -self.crf(lstm_feats, labels, reduction='mean')
            return loss
        else:
            predictions = self.crf.decode(lstm_feats)
            return predictions

In [ ]:
tag_to_ix = {}
embedding_size = 512
hidden_size = 768
epoch_size = 300
model = BiLSTM_CRF(embedding_size, tag_to_ix, hidden_size)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

for epoch in range(epoch_size):
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

